# Data Science Capstone Project

## Week - 1

## Introduction/Business Problem

## Part - 1

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

### Answer:

Where in Rio de Janeiro would it be best to open a restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another?

The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a restaurant in Rio de Janeiro and wants to know if it is a viable option where they intend on opening.

## Data

## Part - 2

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

### Answer:

The data that will be used is Foursquare data for Rio de Janeiro city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011
Libraries imported.


In [2]:
# Get longitude and latitude for Toronto
address = 'Rio de Janeiro, Rio de Janeiro'

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rio de Janeiro is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rio de Janeiro is -22.9110137, -43.2093727.


In [3]:
#Foursquare Credentials
CLIENT_ID = 'WAC2GVD402TM1UPVQNVUQADEUXP2B1KT3T5L4OTSISQSVBA2'
CLIENT_SECRET = '1OC5PGE34ABZJHVRZRIGOY0KGIE220JLKVYHXNEYRYZIXSXV'
VERSION = '20200609'
LIMIT = 100

In [4]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

In [9]:
# assign JSON to venues
venues = results['response']['venues']

# tranforming the venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",False,56e04e1d38fa3ae11155b863,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,RJ,Riá Restaurant,v-1591679595
1,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4ece6782cc219860f521dd5d,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,RJ,Restaurante Pitada,v-1591679595
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,50e0b634e4b0f94bd8a38f47,Navio Soberano,BR,Rio de Janeiro,Brasil,NaN,2849,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.8928508209919...",-22.892851,-43.189795,NaN,RJ,Restaurant El Duero,v-1591679595
3,"[{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...",False,5658ef2a498e0ac7f69bac6a,"Rua Aníbal Benévolo, 132. Cidade Nova",BR,Rio de Janeiro,Brasil,Avenida Salvador De Sá,1087,"[Rua Aníbal Benévolo, 132. Cidade Nova (Avenid...","[{'label': 'display', 'lat': -22.9118662927738...",-22.911866,-43.198804,NaN,RJ,Restaurante Sambódromo,v-1591679595
4,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4fc1101ee4b0557038e08c56,"Rua Melo e Sousa, 122, São Cristovão",BR,Rio de Janeiro,Brasil,NaN,782,"[Rua Melo e Sousa, 122, São Cristovão, Rio de ...","[{'label': 'display', 'lat': -22.9047394071136...",-22.904739,-43.212804,NaN,RJ,Restaurante Cheiro Verde,v-1591679595


In [10]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Riá Restaurant,Bistro,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,RJ,56e04e1d38fa3ae11155b863
1,Restaurante Pitada,Brazilian Restaurant,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,RJ,4ece6782cc219860f521dd5d
2,Restaurant El Duero,Restaurant,Navio Soberano,BR,Rio de Janeiro,Brasil,NaN,2849,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.8928508209919...",-22.892851,-43.189795,NaN,RJ,50e0b634e4b0f94bd8a38f47
3,Restaurante Sambódromo,Buffet,"Rua Aníbal Benévolo, 132. Cidade Nova",BR,Rio de Janeiro,Brasil,Avenida Salvador De Sá,1087,"[Rua Aníbal Benévolo, 132. Cidade Nova (Avenid...","[{'label': 'display', 'lat': -22.9118662927738...",-22.911866,-43.198804,NaN,RJ,5658ef2a498e0ac7f69bac6a
4,Restaurante Cheiro Verde,Brazilian Restaurant,"Rua Melo e Sousa, 122, São Cristovão",BR,Rio de Janeiro,Brasil,NaN,782,"[Rua Melo e Sousa, 122, São Cristovão, Rio de ...","[{'label': 'display', 'lat': -22.9047394071136...",-22.904739,-43.212804,NaN,RJ,4fc1101ee4b0557038e08c56


In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map